# Signal Synchronization


In [ ]:
import warnings

import numpy as np
try:
    import cupy as cp
    NO_CUPY = False
except ImportError:
    warnings.warn("CuPy is not installed (needed for GPU acceleration). Falling back to CPU, which might be extremly slow!")
    NO_CUPY = True
from scipy import signal
import matplotlib.pyplot as plt

from datetime import datetime

import sys
sys.path.append('../')
import rtl_sdr_pr.ioutils
import rtl_sdr_pr.processing

## FIR-based sub-sample Synchronization


In [ ]:
def frac_delay_fir(ntaps, u):
    if u % 1 == 0:
        u += np.finfo(float).eps
    
    N = ntaps - 1
    n = np.linspace(-N/2, N/2, ntaps)
    win = signal.chebwin(ntaps, 70)
    return np.multiply(np.sinc(n - u), win.T)

In [ ]:
win = np.hanning(7)/4
x = np.convolve(win, np.ones(20))
b_zero = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y1 = np.convolve(b_zero, x)

ntaps = 19
b_dly = frac_delay_fir(ntaps, 0.5)
b_adv = frac_delay_fir(ntaps, -0.5)
y2 = np.convolve(b_dly, x)
y3 = np.convolve(b_adv, x)

plt.figure()
plt.plot(x)
plt.grid(True)

plt.figure()
plt.plot(y1)
plt.grid(True)

plt.figure()
plt.plot(y1)
plt.plot(y2)
plt.plot(y3)
plt.grid(True)

plt.figure()
plt.plot(b_dly)
plt.plot(b_adv)
plt.grid(True)

## Correlation-based Synchronization

Use timestamp information (if available) and cross-correlation to find sample offset in recordings. This approach assumes the receiver hardware is phase and frequency coherent (i.e. sharing a common local oscillator).

First some setup-work, which includes reading file headers and determining a rough offset based on recording timestamps.


In [ ]:
CPI = 2 # in seconds

ref_file_path = "../data/pluto_a_ref.2021-07-27T15_24_17_598.sdriq"
surv_file_path = "../data/pluto_b_surv.2021-07-27T15_24_21_819.sdriq"

_, ref_hdr = rtl_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, 0, 0)
_, surv_hdr = rtl_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, 0, 0)

assert ref_hdr["sample_rate"] == surv_hdr["sample_rate"]
assert ref_hdr["center_frequency"] == surv_hdr["center_frequency"]

sample_rate = ref_hdr["sample_rate"]
center_frequency = ref_hdr['center_frequency']
num_samples_in_cpi = CPI * sample_rate

print(f"Sample rate: {sample_rate / 1e6:0.1f} MHz, Center frequency: {center_frequency / 1e6:0.1f} MHz")
print(f"Start of reference channel: {ref_hdr['start_time_stamp']} -> {datetime.fromtimestamp(ref_hdr['start_time_stamp'])}")
print(f"Start of surveillance channel: {surv_hdr['start_time_stamp']} -> {datetime.fromtimestamp(surv_hdr['start_time_stamp'])}")

time_diff = surv_hdr['start_time_stamp'] - ref_hdr['start_time_stamp']
estim_sample_shift = time_diff * sample_rate

print(f"Time delta: {time_diff} sec, Estimated sample shift: {estim_sample_shift}")

ref_samples, _ = rtl_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, int(num_samples_in_cpi * 2), max(0, int(estim_sample_shift - num_samples_in_cpi)))
surv_samples, _ = rtl_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, int(num_samples_in_cpi * 2), max(0, -int(estim_sample_shift - num_samples_in_cpi)))

sub_sample_factor = 1
ref_samples = ref_samples[::sub_sample_factor]
surv_samples = surv_samples[::sub_sample_factor]

Now correlate and find the correlation peak.


In [ ]:
if NO_CUPY:
    # PLEASE INSTALL CUPY! OTHERWISE THIS IS SOOO INCREADIBLY SLOW!
    corr = np.correlate(ref_samples, surv_samples, mode="same")
else:
    corr = cp.correlate(cp.asarray(ref_samples), cp.asarray(surv_samples), mode="same").get()
corr_mag = np.abs(corr)
corr_mag_db = 10 * np.log10(corr_mag / np.max(corr_mag))

peak_idx = np.argmax(corr_mag)
peak_time_offset = (peak_idx - corr_mag.size / 2) / sample_rate * sub_sample_factor

print(f"Correlation peak at {peak_idx} with time offset of {peak_time_offset * 1000:0.3f} msec after initial estimation")

In [ ]:
plt.figure(figsize=(40, 7))
plt.plot(np.linspace(-corr_mag_db.size / 2 / sample_rate * 1000 * sub_sample_factor, corr_mag_db.size / 2 / sample_rate * 1000 * sub_sample_factor, corr_mag_db.size), corr_mag_db)
plt.xlabel("time delta [msec]")
plt.ylabel("correlation [dB]")
plt.xticks(np.arange(-CPI * 1000, CPI * 1000 + 1, step=100, dtype=np.int32))
plt.grid()

plt.annotate(f"Peak\n"
             f"sample offset: {peak_idx * sub_sample_factor}\n"
             f"time offset: {peak_time_offset * 1000:0.3f} msec",
             xy=(peak_time_offset * 1000, corr_mag_db[peak_idx]),
             xytext=(-100, 30),
             xycoords="data",
             textcoords="offset pixels",
             arrowprops={'arrowstyle': 'wedge'});